# GM-VAE for SMB

In [1]:
num_epochs=10000
num_components=2  # only the starting number of components

## Import Libraries

In [2]:
import json

import numpy as np
import argparse
import random
from torchvision import datasets, transforms
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import torch.utils.data
from scipy.io import loadmat

import sys
sys.path.append('/home/yangz2/projects/gmvae_and_gmmvae/pytorch/')
sys.path.append('../../1906-pcgml/pcgml-gmmVae-exp/modules/')
import vglc_with_path_encodings

from model.GMVAE import *
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append('/home/yangz2/projects/modules/')
from log_training_progress import email_training_progress

## Load data

In [3]:
def open_json_as_nparray(json_fname):
    with open(json_fname, 'r') as json_f:
        return np.array(json.load(json_f))
    
chunks_int = open_json_as_nparray('/home/yangz2/projects/gmvae_and_gmmvae/data/smba_chunks_int.json')
chunks_onehot = np.eye(len(np.unique(chunks_int)))[chunks_int]
print(chunks_onehot.shape)

chunks_onehot = chunks_onehot.reshape(
    chunks_onehot.shape[0], 
    chunks_onehot.shape[1] * chunks_onehot.shape[2] * chunks_onehot.shape[3]
)
print(chunks_onehot.shape)

(2698, 16, 16, 12)
(2698, 3072)


In [4]:
def get_random_labels(num_components, size):
    return np.random.randint(num_components, size=size)

## Input Parameters

In [5]:
num_components = 8

In [6]:
########################################################
## Input Parameters
#########################################################
parser = argparse.ArgumentParser(description='PyTorch Implementation of DGM Clustering')

## Used only in notebooks
parser.add_argument('-f', '--file',
                    help='Path for input file. First line should contain number of lines to search in')

## Dataset
parser.add_argument('--dataset', type=str, choices=['mnist'],
                    default='mnist', help='dataset (default: mnist)')
parser.add_argument('--seed', type=int, default=1, help='random seed (default: 1)')

## GPU
parser.add_argument('--cuda', type=int, default=1,
                    help='use of cuda (default: 1)')
parser.add_argument('--gpuID', type=int, default=0,
                    help='set gpu id to use (default: 0)')

## Training
parser.add_argument('--epochs', type=int, default=num_epochs,
                    help='number of total epochs to run (default: 200)')
parser.add_argument('--batch_size', default=64, type=int,
                    help='mini-batch size (default: 64)')
parser.add_argument('--batch_size_val', default=200, type=int,
                    help='mini-batch size of validation (default: 200)')
parser.add_argument('--learning_rate', default=1e-3, type=float,
                    help='learning rate (default: 0.001)')
parser.add_argument('--decay_epoch', default=-1, type=int, 
                    help='Reduces the learning rate every decay_epoch')
parser.add_argument('--lr_decay', default=0.5, type=float,
                    help='Learning rate decay for training (default: 0.5)')

## Architecture
parser.add_argument('--num_classes', type=int, default=num_components,
                    help='number of classes (default: 10)')  # edited for this task
parser.add_argument('--gaussian_size', default=64, type=int,
                    help='gaussian size (default: 64)')
parser.add_argument('--input_size', default=3072, type=int,
                    help='input size (default: 784)')  # edited for this task

## Partition parameters
parser.add_argument('--train_proportion', default=1, type=float,
                    help='proportion of examples to consider for training only (default: 1.0)')

## Gumbel parameters
parser.add_argument('--init_temp', default=1.0, type=float,
                    help='Initial temperature used in gumbel-softmax (recommended 0.5-1.0, default:1.0)')
parser.add_argument('--decay_temp', default=1, type=int, 
                    help='Set 1 to decay gumbel temperature at every epoch (default: 1)')
parser.add_argument('--hard_gumbel', default=0, type=int, 
                    help='Set 1 to use the hard version of gumbel-softmax (default: 1)')
parser.add_argument('--min_temp', default=0.5, type=float, 
                    help='Minimum temperature of gumbel-softmax after annealing (default: 0.5)' )
parser.add_argument('--decay_temp_rate', default=0.013862944, type=float,
                    help='Temperature decay rate at every epoch (default: 0.013862944)')

## Loss function parameters
parser.add_argument('--w_gauss', default=2, type=float,
                    help='weight of gaussian loss (default: 1)')
parser.add_argument('--w_categ', default=1, type=float,
                    help='weight of categorical loss (default: 1)')
parser.add_argument('--w_rec', default=1, type=float,
                    help='weight of reconstruction loss (default: 1)')
parser.add_argument('--rec_type', type=str, choices=['bce', 'mse'],
                    default='bce', help='desired reconstruction loss function (default: bce)')

## Others
parser.add_argument('--verbose', default=0, type=int,
                    help='print extra information at every epoch.(default: 0)')

args = parser.parse_args()

Set random seed in case it was specified in the parameters


In [7]:
## Random Seed
SEED = args.seed
np.random.seed(SEED)
random.seed(SEED)
torch.manual_seed(SEED)
if args.cuda:
    torch.cuda.manual_seed(SEED)

## Data Partition

We split the training data into train and validation according to the *train_proportion* parameter:

In [8]:
def get_train_dl_and_valid_dl(chunks, labels):

    valid_indices = np.random.randint(len(chunks), size=100)
    train_chunks, valid_chunks = chunks, chunks[valid_indices]
    train_labels, valid_labels = labels, labels[valid_indices]

    train_ds = TensorDataset(torch.from_numpy(train_chunks).float(), torch.from_numpy(train_labels).long())
    valid_ds = TensorDataset(torch.from_numpy(valid_chunks).float(), torch.from_numpy(valid_labels).long())

    train_dl = DataLoader(train_ds, batch_size=args.batch_size, shuffle=True)
    valid_dl = DataLoader(valid_ds, batch_size=args.batch_size_val, shuffle=False)
    
    return train_dl, valid_dl

## Train Model

In [ ]:
email_training_progress(f'Training session has started!')
for num_components in [2, 40]:
    
    args.num_classes = num_components
    gmvae = GMVAE(args)
    
    labels = get_random_labels(num_components, len(chunks_onehot))
    train_dl, valid_dl = get_train_dl_and_valid_dl(chunks_onehot, labels)
    gmvae.train(train_dl, valid_dl)
    
    torch.save(
        gmvae.network.state_dict(),
        f'../qualitative_clustering_analysis/smba_gmvae_{num_components}_{num_epochs}.pth'
    )
    
    email_training_progress(f'GM-VAE with {num_components} components has finished training!')
email_training_progress('Shutdown VM now!')

(Epoch 1 / 10000) Train_Loss: 465.215; Val_Loss: 232.649   Train_ACC: 50.037; Val_ACC: 57.000   Train_NMI: 0.004; Val_NMI: 0.000
(Epoch 2 / 10000) Train_Loss: 231.949; Val_Loss: 225.933   Train_ACC: 50.111; Val_ACC: 57.000   Train_NMI: 0.000; Val_NMI: 0.000
(Epoch 3 / 10000) Train_Loss: 227.843; Val_Loss: 223.958   Train_ACC: 50.111; Val_ACC: 57.000   Train_NMI: 0.000; Val_NMI: 0.000
(Epoch 4 / 10000) Train_Loss: 222.196; Val_Loss: 217.976   Train_ACC: 50.111; Val_ACC: 57.000   Train_NMI: 0.000; Val_NMI: 0.000
(Epoch 5 / 10000) Train_Loss: 217.263; Val_Loss: 215.247   Train_ACC: 50.111; Val_ACC: 57.000   Train_NMI: 0.000; Val_NMI: 0.000
(Epoch 6 / 10000) Train_Loss: 212.845; Val_Loss: 211.994   Train_ACC: 50.111; Val_ACC: 57.000   Train_NMI: 0.000; Val_NMI: 0.000
(Epoch 7 / 10000) Train_Loss: 210.005; Val_Loss: 205.234   Train_ACC: 50.111; Val_ACC: 57.000   Train_NMI: 0.000; Val_NMI: 0.000
(Epoch 8 / 10000) Train_Loss: 205.825; Val_Loss: 203.846   Train_ACC: 50.111; Val_ACC: 57.000   T

(Epoch 65 / 10000) Train_Loss: 135.976; Val_Loss: 140.612   Train_ACC: 50.111; Val_ACC: 57.000   Train_NMI: 0.000; Val_NMI: 0.000
(Epoch 66 / 10000) Train_Loss: 136.145; Val_Loss: 139.770   Train_ACC: 50.111; Val_ACC: 57.000   Train_NMI: 0.000; Val_NMI: 0.000
(Epoch 67 / 10000) Train_Loss: 134.896; Val_Loss: 137.447   Train_ACC: 50.111; Val_ACC: 57.000   Train_NMI: 0.000; Val_NMI: 0.000
(Epoch 68 / 10000) Train_Loss: 134.570; Val_Loss: 135.549   Train_ACC: 50.111; Val_ACC: 57.000   Train_NMI: 0.000; Val_NMI: 0.000
(Epoch 69 / 10000) Train_Loss: 133.833; Val_Loss: 136.561   Train_ACC: 50.111; Val_ACC: 57.000   Train_NMI: 0.000; Val_NMI: 0.000
(Epoch 70 / 10000) Train_Loss: 133.606; Val_Loss: 138.624   Train_ACC: 50.111; Val_ACC: 57.000   Train_NMI: 0.000; Val_NMI: 0.000
(Epoch 71 / 10000) Train_Loss: 133.251; Val_Loss: 134.763   Train_ACC: 50.111; Val_ACC: 57.000   Train_NMI: 0.000; Val_NMI: 0.000
(Epoch 72 / 10000) Train_Loss: 132.478; Val_Loss: 134.904   Train_ACC: 50.111; Val_ACC: 57

(Epoch 258 / 10000) Train_Loss: 65.008; Val_Loss: 61.782   Train_ACC: 50.111; Val_ACC: 57.000   Train_NMI: 0.000; Val_NMI: 0.000
(Epoch 259 / 10000) Train_Loss: 65.571; Val_Loss: 64.292   Train_ACC: 50.111; Val_ACC: 57.000   Train_NMI: 0.000; Val_NMI: 0.000
(Epoch 260 / 10000) Train_Loss: 65.209; Val_Loss: 63.980   Train_ACC: 50.111; Val_ACC: 57.000   Train_NMI: 0.000; Val_NMI: 0.000
(Epoch 261 / 10000) Train_Loss: 65.431; Val_Loss: 62.268   Train_ACC: 50.111; Val_ACC: 57.000   Train_NMI: 0.000; Val_NMI: 0.000
(Epoch 262 / 10000) Train_Loss: 65.592; Val_Loss: 61.612   Train_ACC: 50.111; Val_ACC: 57.000   Train_NMI: 0.000; Val_NMI: 0.000
(Epoch 263 / 10000) Train_Loss: 65.273; Val_Loss: 61.923   Train_ACC: 50.111; Val_ACC: 57.000   Train_NMI: 0.000; Val_NMI: 0.000
(Epoch 264 / 10000) Train_Loss: 64.065; Val_Loss: 62.796   Train_ACC: 50.111; Val_ACC: 57.000   Train_NMI: 0.000; Val_NMI: 0.000
(Epoch 265 / 10000) Train_Loss: 63.993; Val_Loss: 62.605   Train_ACC: 50.111; Val_ACC: 57.000   T

(Epoch 322 / 10000) Train_Loss: 57.979; Val_Loss: 54.078   Train_ACC: 50.111; Val_ACC: 57.000   Train_NMI: 0.000; Val_NMI: 0.000
(Epoch 323 / 10000) Train_Loss: 56.910; Val_Loss: 56.118   Train_ACC: 50.111; Val_ACC: 57.000   Train_NMI: 0.000; Val_NMI: 0.000
(Epoch 324 / 10000) Train_Loss: 55.651; Val_Loss: 53.173   Train_ACC: 50.111; Val_ACC: 57.000   Train_NMI: 0.000; Val_NMI: 0.000
(Epoch 325 / 10000) Train_Loss: 55.825; Val_Loss: 53.569   Train_ACC: 50.111; Val_ACC: 57.000   Train_NMI: 0.000; Val_NMI: 0.000
(Epoch 326 / 10000) Train_Loss: 56.303; Val_Loss: 55.344   Train_ACC: 50.111; Val_ACC: 57.000   Train_NMI: 0.000; Val_NMI: 0.000
(Epoch 327 / 10000) Train_Loss: 57.563; Val_Loss: 56.729   Train_ACC: 50.111; Val_ACC: 57.000   Train_NMI: 0.000; Val_NMI: 0.000
(Epoch 328 / 10000) Train_Loss: 58.471; Val_Loss: 55.541   Train_ACC: 50.111; Val_ACC: 57.000   Train_NMI: 0.000; Val_NMI: 0.000
(Epoch 329 / 10000) Train_Loss: 57.801; Val_Loss: 54.776   Train_ACC: 50.111; Val_ACC: 57.000   T

(Epoch 386 / 10000) Train_Loss: 53.414; Val_Loss: 49.572   Train_ACC: 50.111; Val_ACC: 57.000   Train_NMI: 0.000; Val_NMI: 0.000
(Epoch 387 / 10000) Train_Loss: 52.191; Val_Loss: 51.082   Train_ACC: 50.111; Val_ACC: 57.000   Train_NMI: 0.000; Val_NMI: 0.000
(Epoch 388 / 10000) Train_Loss: 51.542; Val_Loss: 48.512   Train_ACC: 50.111; Val_ACC: 57.000   Train_NMI: 0.000; Val_NMI: 0.000
(Epoch 389 / 10000) Train_Loss: 52.315; Val_Loss: 48.804   Train_ACC: 50.111; Val_ACC: 57.000   Train_NMI: 0.000; Val_NMI: 0.000
(Epoch 390 / 10000) Train_Loss: 50.532; Val_Loss: 49.562   Train_ACC: 50.111; Val_ACC: 57.000   Train_NMI: 0.000; Val_NMI: 0.000
(Epoch 391 / 10000) Train_Loss: 50.490; Val_Loss: 50.153   Train_ACC: 50.111; Val_ACC: 57.000   Train_NMI: 0.000; Val_NMI: 0.000
(Epoch 392 / 10000) Train_Loss: 50.841; Val_Loss: 49.442   Train_ACC: 50.111; Val_ACC: 57.000   Train_NMI: 0.000; Val_NMI: 0.000
(Epoch 393 / 10000) Train_Loss: 51.203; Val_Loss: 49.783   Train_ACC: 50.111; Val_ACC: 57.000   T

...

(Epoch 2214 / 10000) Train_Loss: 25.080; Val_Loss: 25.266   Train_ACC: 7.079; Val_ACC: 34.000   Train_NMI: 9.410; Val_NMI: 65.296
(Epoch 2215 / 10000) Train_Loss: 24.817; Val_Loss: 24.042   Train_ACC: 7.042; Val_ACC: 33.000   Train_NMI: 9.509; Val_NMI: 64.710
(Epoch 2216 / 10000) Train_Loss: 25.673; Val_Loss: 24.059   Train_ACC: 6.820; Val_ACC: 34.000   Train_NMI: 9.553; Val_NMI: 65.064
(Epoch 2217 / 10000) Train_Loss: 24.957; Val_Loss: 23.756   Train_ACC: 6.931; Val_ACC: 31.000   Train_NMI: 9.272; Val_NMI: 64.200
(Epoch 2218 / 10000) Train_Loss: 25.276; Val_Loss: 23.915   Train_ACC: 6.857; Val_ACC: 34.000   Train_NMI: 9.330; Val_NMI: 65.374
(Epoch 2219 / 10000) Train_Loss: 25.324; Val_Loss: 24.661   Train_ACC: 7.005; Val_ACC: 33.000   Train_NMI: 9.368; Val_NMI: 65.407
(Epoch 2220 / 10000) Train_Loss: 24.737; Val_Loss: 24.526   Train_ACC: 6.931; Val_ACC: 34.000   Train_NMI: 9.317; Val_NMI: 65.443
(Epoch 2221 / 10000) Train_Loss: 25.566; Val_Loss: 27.082   Train_ACC: 7.005; Val_ACC: 35.

(Epoch 2278 / 10000) Train_Loss: 28.987; Val_Loss: 26.778   Train_ACC: 7.116; Val_ACC: 33.000   Train_NMI: 9.803; Val_NMI: 65.605
(Epoch 2279 / 10000) Train_Loss: 28.492; Val_Loss: 28.156   Train_ACC: 6.968; Val_ACC: 33.000   Train_NMI: 9.246; Val_NMI: 65.252
(Epoch 2280 / 10000) Train_Loss: 26.673; Val_Loss: 24.566   Train_ACC: 7.005; Val_ACC: 34.000   Train_NMI: 9.385; Val_NMI: 65.561
(Epoch 2281 / 10000) Train_Loss: 25.892; Val_Loss: 26.445   Train_ACC: 6.931; Val_ACC: 34.000   Train_NMI: 9.499; Val_NMI: 65.390
(Epoch 2282 / 10000) Train_Loss: 25.581; Val_Loss: 25.558   Train_ACC: 6.968; Val_ACC: 35.000   Train_NMI: 9.667; Val_NMI: 65.547
(Epoch 2283 / 10000) Train_Loss: 32.379; Val_Loss: 30.204   Train_ACC: 7.153; Val_ACC: 35.000   Train_NMI: 9.626; Val_NMI: 66.445
(Epoch 2284 / 10000) Train_Loss: 30.523; Val_Loss: 27.172   Train_ACC: 6.820; Val_ACC: 34.000   Train_NMI: 9.143; Val_NMI: 65.937
(Epoch 2285 / 10000) Train_Loss: 27.107; Val_Loss: 25.680   Train_ACC: 7.153; Val_ACC: 34.

(Epoch 2342 / 10000) Train_Loss: 25.259; Val_Loss: 24.608   Train_ACC: 6.968; Val_ACC: 34.000   Train_NMI: 9.309; Val_NMI: 65.587
(Epoch 2343 / 10000) Train_Loss: 24.468; Val_Loss: 25.855   Train_ACC: 6.857; Val_ACC: 33.000   Train_NMI: 9.518; Val_NMI: 65.495
(Epoch 2344 / 10000) Train_Loss: 24.570; Val_Loss: 26.665   Train_ACC: 7.079; Val_ACC: 34.000   Train_NMI: 9.512; Val_NMI: 66.043
(Epoch 2345 / 10000) Train_Loss: 25.822; Val_Loss: 26.221   Train_ACC: 7.005; Val_ACC: 36.000   Train_NMI: 9.483; Val_NMI: 66.627
(Epoch 2346 / 10000) Train_Loss: 24.814; Val_Loss: 23.755   Train_ACC: 7.005; Val_ACC: 35.000   Train_NMI: 9.618; Val_NMI: 65.937
(Epoch 2347 / 10000) Train_Loss: 25.281; Val_Loss: 26.113   Train_ACC: 7.005; Val_ACC: 35.000   Train_NMI: 9.631; Val_NMI: 66.043
(Epoch 2348 / 10000) Train_Loss: 26.107; Val_Loss: 26.985   Train_ACC: 7.191; Val_ACC: 33.000   Train_NMI: 9.304; Val_NMI: 65.820
(Epoch 2349 / 10000) Train_Loss: 25.353; Val_Loss: 26.154   Train_ACC: 7.042; Val_ACC: 33.

(Epoch 2406 / 10000) Train_Loss: 27.455; Val_Loss: 25.033   Train_ACC: 7.079; Val_ACC: 34.000   Train_NMI: 9.387; Val_NMI: 65.665
(Epoch 2407 / 10000) Train_Loss: 25.802; Val_Loss: 24.721   Train_ACC: 6.857; Val_ACC: 30.000   Train_NMI: 9.435; Val_NMI: 64.420
(Epoch 2408 / 10000) Train_Loss: 27.245; Val_Loss: 26.929   Train_ACC: 7.116; Val_ACC: 34.000   Train_NMI: 9.602; Val_NMI: 66.772
(Epoch 2409 / 10000) Train_Loss: 27.315; Val_Loss: 25.776   Train_ACC: 6.931; Val_ACC: 33.000   Train_NMI: 9.451; Val_NMI: 65.705
(Epoch 2410 / 10000) Train_Loss: 25.158; Val_Loss: 25.074   Train_ACC: 7.005; Val_ACC: 34.000   Train_NMI: 9.394; Val_NMI: 66.017
(Epoch 2411 / 10000) Train_Loss: 25.716; Val_Loss: 24.729   Train_ACC: 7.079; Val_ACC: 36.000   Train_NMI: 9.546; Val_NMI: 66.222
(Epoch 2412 / 10000) Train_Loss: 28.248; Val_Loss: 27.170   Train_ACC: 7.005; Val_ACC: 34.000   Train_NMI: 9.434; Val_NMI: 65.704
(Epoch 2413 / 10000) Train_Loss: 27.489; Val_Loss: 27.590   Train_ACC: 7.079; Val_ACC: 38.

(Epoch 2470 / 10000) Train_Loss: 24.679; Val_Loss: 23.452   Train_ACC: 7.116; Val_ACC: 34.000   Train_NMI: 9.493; Val_NMI: 66.066
(Epoch 2471 / 10000) Train_Loss: 25.297; Val_Loss: 24.855   Train_ACC: 7.228; Val_ACC: 34.000   Train_NMI: 9.548; Val_NMI: 65.937
(Epoch 2472 / 10000) Train_Loss: 26.199; Val_Loss: 26.370   Train_ACC: 7.153; Val_ACC: 35.000   Train_NMI: 9.356; Val_NMI: 66.484
(Epoch 2473 / 10000) Train_Loss: 25.788; Val_Loss: 26.304   Train_ACC: 7.042; Val_ACC: 34.000   Train_NMI: 9.590; Val_NMI: 66.156
(Epoch 2474 / 10000) Train_Loss: 25.600; Val_Loss: 23.616   Train_ACC: 7.153; Val_ACC: 34.000   Train_NMI: 9.675; Val_NMI: 66.302
(Epoch 2475 / 10000) Train_Loss: 25.232; Val_Loss: 24.414   Train_ACC: 7.005; Val_ACC: 34.000   Train_NMI: 9.496; Val_NMI: 65.630
(Epoch 2476 / 10000) Train_Loss: 26.560; Val_Loss: 26.251   Train_ACC: 6.894; Val_ACC: 33.000   Train_NMI: 9.591; Val_NMI: 65.326
(Epoch 2477 / 10000) Train_Loss: 28.671; Val_Loss: 26.772   Train_ACC: 7.005; Val_ACC: 31.